In [88]:
%matplotlib inline
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from random import shuffle
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
import warnings
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [89]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        for key, value in  sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fiin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))

        return self.sentences

    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

In [90]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def text_to_wordlist(text, remove_stopwords=True, stem_words=False):    
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [wordnet_lemmatizer.lemmatize(w) for w in text if not w in stops ]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    
    text = re.sub(r"\<", " ", text)
    text = re.sub(r"\>", " ", text)
    
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [91]:
sources = {'../data/notebook3/test_neg.txt':'TEST_NEG', '../data/notebook3/test_pos.txt':'TEST_POS', '../data/notebook3/train_neg.txt':'TRAIN_NEG', '../data/notebook3/train_pos.txt':'TRAIN_POS'}

In [92]:
train_data = []
test_data = []
train_labels = []
test_labels = []
words = []
for source, prefix in sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line_ in enumerate(fin):
                        line = text_to_wordlist(str(line_,'utf-8'))
                        for i in line.split():
                            words.append(i)
                        if prefix is 'TRAIN_POS':
                            train_data.append(line)
                            train_labels.append(1)
                        elif prefix is 'TRAIN_NEG':
                            train_data.append(line)
                            train_labels.append(0)
                        elif prefix is 'TEST_POS':
                            test_data.append(line)
                            test_labels.append(1)
                        else:
                            test_data.append(line)
                            test_labels.append(0)

In [93]:
W = np.memmap("/home/savitru/deep_learning/data/embed.dat", dtype=np.double, mode="r", shape=(3000000, 300))
with open("/home/savitru/deep_learning/data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())

vocab_dict = {w: k for k, w in enumerate(vocab_list)}

In [94]:
print(len(words))

111351


In [95]:
windows_size = 10
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_data+test_data)
tokenizer.fit_on_sequences(train_data+test_data)

X = tokenizer.texts_to_sequences(train_data+test_data)

(142,)


In [96]:
vocab_size = len(words)
print(vocab_size)
def fill_embedding_matrix_google(vocab_size, tokenizer):  
    embedding_matrix = np.zeros((vocab_size+1, 300))  
    for word, i in tokenizer.word_index.items():
        if word in vocab_dict:
            embedding_matrix[i] = vocab_dict[word]        
    return embedding_matrix

111351


In [97]:
embedding_matrix_gg = fill_embedding_matrix_google(vocab_size, tokenizer)

In [98]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = windows_size


embedding_layer = Embedding(vocab_size+1,
        EMBEDDING_DIM,
        weights=[embedding_matrix_gg],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)

In [101]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

padded_docs = pad_sequences(X, maxlen=10, padding='post')

In [102]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, train_labels+test_labels , test_size=0.2, random_state=42)

In [103]:
print(padded_docs.shape)

(142, 10)


In [106]:
from sklearn.utils import class_weight
from keras.models import Sequential,Model
from keras.layers import LSTM, Dense, Flatten, Dropout, Bidirectional, concatenate, Input
from keras.utils import to_categorical
from keras import optimizers
import keras.backend as K

model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(300, return_sequences=True, recurrent_dropout=0.2,
                   dropout=0.1), merge_mode='concat'))
model.add(Bidirectional(LSTM(300, return_sequences=False, recurrent_dropout=0.2), merge_mode='concat'))

model.add(Dense(1, activation='sigmoid'))

sgd = optimizers.SGD(lr=0.1, momentum=0.8499999998, decay=0, nesterov=True)

model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy','mae','mse'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 300)           33405600  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 10, 600)           1442400   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 600)               2162400   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 601       
Total params: 37,011,001
Trainable params: 3,605,401
Non-trainable params: 33,405,600
_________________________________________________________________
None


In [109]:
model.fit(X_train, y_train, epochs=300, verbose=1)

acc, mae, mse = model.evaluate(X_test, y_test, verbose=1)

print('Accu ',acc)
print(' mae ',mae)
print(' mse ',mse)

Epoch 1/300
113/113 [==============================] - 1s 8ms/step - loss: 0.6233 - acc: 0.6903 - mean_absolute_error: 0.4398 - mean_squared_error: 0.2154
Epoch 2/300
113/113 [==============================] - 1s 8ms/step - loss: 0.6152 - acc: 0.6903 - mean_absolute_error: 0.4123 - mean_squared_error: 0.2122
Epoch 3/300
113/113 [==============================] - 1s 9ms/step - loss: 0.6107 - acc: 0.6903 - mean_absolute_error: 0.4183 - mean_squared_error: 0.2102
Epoch 4/300
113/113 [==============================] - 1s 8ms/step - loss: 0.6137 - acc: 0.6903 - mean_absolute_error: 0.4214 - mean_squared_error: 0.2113
Epoch 5/300
113/113 [==============================] - 1s 7ms/step - loss: 0.6238 - acc: 0.6814 - mean_absolute_error: 0.4541 - mean_squared_error: 0.2161
Epoch 6/300
113/113 [==============================] - 1s 7ms/step - loss: 0.6201 - acc: 0.6903 - mean_absolute_error: 0.4012 - mean_squared_error: 0.2139
Epoch 7/300
113/113 [==============================] - 1s 7ms/step - l

113/113 [==============================] - 1s 8ms/step - loss: 0.6152 - acc: 0.6991 - mean_absolute_error: 0.4126 - mean_squared_error: 0.2107
Epoch 54/300
113/113 [==============================] - 1s 8ms/step - loss: 0.6015 - acc: 0.6903 - mean_absolute_error: 0.4127 - mean_squared_error: 0.2058
Epoch 55/300
113/113 [==============================] - 1s 9ms/step - loss: 0.6146 - acc: 0.6991 - mean_absolute_error: 0.4293 - mean_squared_error: 0.2115
Epoch 56/300
113/113 [==============================] - 1s 9ms/step - loss: 0.6086 - acc: 0.6903 - mean_absolute_error: 0.3877 - mean_squared_error: 0.2085
Epoch 57/300
113/113 [==============================] - 1s 7ms/step - loss: 0.6143 - acc: 0.6814 - mean_absolute_error: 0.4355 - mean_squared_error: 0.2116
Epoch 58/300
113/113 [==============================] - 1s 7ms/step - loss: 0.5975 - acc: 0.7168 - mean_absolute_error: 0.4169 - mean_squared_error: 0.2041
Epoch 59/300
113/113 [==============================] - 1s 8ms/step - loss: 0

113/113 [==============================] - 1s 7ms/step - loss: 0.6224 - acc: 0.6991 - mean_absolute_error: 0.4205 - mean_squared_error: 0.2144
Epoch 106/300
113/113 [==============================] - 1s 8ms/step - loss: 0.6104 - acc: 0.6991 - mean_absolute_error: 0.3932 - mean_squared_error: 0.2087
Epoch 107/300
113/113 [==============================] - 1s 7ms/step - loss: 0.5942 - acc: 0.6903 - mean_absolute_error: 0.4138 - mean_squared_error: 0.2026
Epoch 108/300
113/113 [==============================] - 1s 7ms/step - loss: 0.5961 - acc: 0.6991 - mean_absolute_error: 0.4109 - mean_squared_error: 0.2032
Epoch 109/300
113/113 [==============================] - 1s 7ms/step - loss: 0.6068 - acc: 0.7080 - mean_absolute_error: 0.4127 - mean_squared_error: 0.2081
Epoch 110/300
113/113 [==============================] - 1s 7ms/step - loss: 0.6053 - acc: 0.7257 - mean_absolute_error: 0.4202 - mean_squared_error: 0.2074
Epoch 111/300
113/113 [==============================] - 1s 7ms/step - l

113/113 [==============================] - 1s 7ms/step - loss: 0.6000 - acc: 0.7168 - mean_absolute_error: 0.4000 - mean_squared_error: 0.2052
Epoch 158/300
113/113 [==============================] - 1s 7ms/step - loss: 0.6079 - acc: 0.6991 - mean_absolute_error: 0.3947 - mean_squared_error: 0.2082
Epoch 159/300
113/113 [==============================] - 1s 7ms/step - loss: 0.5974 - acc: 0.7080 - mean_absolute_error: 0.4082 - mean_squared_error: 0.2044
Epoch 160/300
113/113 [==============================] - 1s 7ms/step - loss: 0.6021 - acc: 0.6991 - mean_absolute_error: 0.4281 - mean_squared_error: 0.2064
Epoch 161/300
113/113 [==============================] - 1s 7ms/step - loss: 0.5932 - acc: 0.7168 - mean_absolute_error: 0.4133 - mean_squared_error: 0.2024
Epoch 162/300
113/113 [==============================] - 1s 7ms/step - loss: 0.5970 - acc: 0.7257 - mean_absolute_error: 0.3976 - mean_squared_error: 0.2040
Epoch 163/300
113/113 [==============================] - 1s 7ms/step - l

113/113 [==============================] - 1s 8ms/step - loss: 0.5946 - acc: 0.7080 - mean_absolute_error: 0.4052 - mean_squared_error: 0.2033
Epoch 210/300
113/113 [==============================] - 1s 9ms/step - loss: 0.6037 - acc: 0.6991 - mean_absolute_error: 0.4090 - mean_squared_error: 0.2069
Epoch 211/300
113/113 [==============================] - 1s 9ms/step - loss: 0.5946 - acc: 0.7168 - mean_absolute_error: 0.4049 - mean_squared_error: 0.2032
Epoch 212/300
113/113 [==============================] - 1s 8ms/step - loss: 0.5965 - acc: 0.6991 - mean_absolute_error: 0.4189 - mean_squared_error: 0.2042
Epoch 213/300
113/113 [==============================] - 1s 10ms/step - loss: 0.6098 - acc: 0.7168 - mean_absolute_error: 0.3893 - mean_squared_error: 0.2088
Epoch 214/300
113/113 [==============================] - 1s 11ms/step - loss: 0.5941 - acc: 0.6991 - mean_absolute_error: 0.4099 - mean_squared_error: 0.2031
Epoch 215/300
113/113 [==============================] - 1s 8ms/step -

113/113 [==============================] - 1s 10ms/step - loss: 0.5951 - acc: 0.7257 - mean_absolute_error: 0.4047 - mean_squared_error: 0.2036
Epoch 262/300
113/113 [==============================] - 2s 15ms/step - loss: 0.5945 - acc: 0.7168 - mean_absolute_error: 0.4126 - mean_squared_error: 0.2032
Epoch 263/300
113/113 [==============================] - 1s 11ms/step - loss: 0.5992 - acc: 0.6991 - mean_absolute_error: 0.3969 - mean_squared_error: 0.2054
Epoch 264/300
113/113 [==============================] - 1s 7ms/step - loss: 0.5976 - acc: 0.7168 - mean_absolute_error: 0.3879 - mean_squared_error: 0.2043
Epoch 265/300
113/113 [==============================] - 1s 9ms/step - loss: 0.5915 - acc: 0.7168 - mean_absolute_error: 0.4141 - mean_squared_error: 0.2020
Epoch 266/300
113/113 [==============================] - 1s 8ms/step - loss: 0.5959 - acc: 0.7080 - mean_absolute_error: 0.4269 - mean_squared_error: 0.2038
Epoch 267/300
113/113 [==============================] - 1s 9ms/step 

ValueError: too many values to unpack (expected 3)